# 如何创建工具
在构建[智能体](/docs/concepts/agents/)时，您需要为其提供一组可用的[工具集](/docs/concepts/tools/)。除了实际被调用的功能函数外，每个工具还包含多个组成部分：
| 属性         | 类型                            | 描述                                                                                                                                                                          || 参数               | 类型                             | 描述                                                                                                                                                                           |
|----------------|----------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|| 名称          | 字符串                             | 在提供给LLM或智能体的一组工具中必须保持唯一。                                                                                                              || 描述         | 字符串                          | 说明该工具的功能。作为LLM（大语言模型）或智能体的上下文信息使用。                                                                                                             || args_schema   | pydantic.BaseModel | 可选但推荐使用，如果使用回调处理程序则必须提供。可用于为预期参数提供更多信息（例如少样本示例）或进行验证。 || return_direct | boolean                         | 仅适用于代理。当设置为True时，代理在调用指定工具后将立即停止，并直接将结果返回给用户。                                             |
LangChain支持从以下方式创建工具：
1. 函数；2. LangChain [可运行对象](/docs/concepts/runnables);3. 通过继承[BaseTool](https://python.langchain.com/api_reference/core/tools/langchain_core.tools.base.BaseTool.html)实现 —— 这是最灵活的方法，它能提供最大程度的控制，但需要付出更多努力和编写更多代码。
通过函数创建工具可能足以满足大多数用例需求，这可以通过简单的 [@tool 装饰器](https://python.langchain.com/api_reference/core/tools/langchain_core.tools.convert.tool.html) 实现。若需要更多配置（例如需同时指定同步和异步实现），也可使用 [StructuredTool.from_function](https://python.langchain.com/api_reference/core/tools/langchain_core.tools.structured.StructuredTool.html#langchain_core.tools.structured.StructuredTool.from_function) 类方法。
在本指南中，我们将概述这些方法。
:::提示
如果工具的名称、描述和JSON模式选择得当，模型的性能会更好。好的，请提供需要翻译的英文文本，我会按照标准Markdown格式将其翻译成中文并直接输出内容。

## 从函数创建工具
### @tool 装饰器
这个 `@tool` 装饰器是定义自定义工具的最简单方式。默认情况下，装饰器会使用函数名称作为工具名称，但也可以通过传递字符串作为第一个参数来覆盖此行为。此外，装饰器会将函数的文档字符串作为工具的描述——因此必须提供文档字符串。

In [1]:
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


或者创建一个**异步**实现，如下所示：

In [2]:
from langchain_core.tools import tool


@tool
async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

请注意，`@tool` 支持解析注解、嵌套模式及其他特性：

In [3]:
from typing import Annotated, List


@tool
def multiply_by_max(
    a: Annotated[int, "scale factor"],
    b: Annotated[List[int], "list of ints over which to take maximum"],
) -> int:
    """Multiply a by the maximum of b."""
    return a * max(b)


print(multiply_by_max.args_schema.model_json_schema())

{'description': 'Multiply a by the maximum of b.',
 'properties': {'a': {'description': 'scale factor',
   'title': 'A',
   'type': 'string'},
  'b': {'description': 'list of ints over which to take maximum',
   'items': {'type': 'integer'},
   'title': 'B',
   'type': 'array'}},
 'required': ['a', 'b'],
 'title': 'multiply_by_maxSchema',
 'type': 'object'}

你也可以通过将工具名称和 JSON 参数传入工具装饰器来自定义它们。

In [4]:
from pydantic import BaseModel, Field


class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


@tool("multiplication-tool", args_schema=CalculatorInput, return_direct=True)
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

multiplication-tool
Multiply two numbers.
{'a': {'description': 'first number', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'second number', 'title': 'B', 'type': 'integer'}}
True


#### 文档字符串解析

`@tool` 可选择性地解析 [Google 风格文档字符串](https://google.github.io/styleguide/pyguide.html#383-functions-and-methods)，并将文档字符串的组成部分（如参数描述）关联到工具模式的相关部分。要切换此行为，请指定 `parse_docstring`：

In [5]:
@tool(parse_docstring=True)
def foo(bar: str, baz: int) -> str:
    """The foo.

    Args:
        bar: The bar.
        baz: The baz.
    """
    return bar


print(foo.args_schema.model_json_schema())

{'description': 'The foo.',
 'properties': {'bar': {'description': 'The bar.',
   'title': 'Bar',
   'type': 'string'},
  'baz': {'description': 'The baz.', 'title': 'Baz', 'type': 'integer'}},
 'required': ['bar', 'baz'],
 'title': 'fooSchema',
 'type': 'object'}

:::注意默认情况下，如果文档字符串解析失败，`@tool(parse_docstring=True)` 会抛出 `ValueError`。详情及示例请参阅 [API 参考文档](https://python.langchain.com/api_reference/core/tools/langchain_core.tools.convert.tool.html)。好的,我会按照要求进行翻译,只输出翻译后的中文内容,并保持markdown格式一致。以下是一个示例:

# 欢迎使用翻译助手

这是一个**markdown格式**的翻译示例。

## 主要功能

- 准确翻译英文到中文
- 保持原始markdown格式
- 支持多种文档元素:
  * 标题
  * 列表
  * 代码块
  * 表格

### 表格示例

| 项目       | 描述          |
|------------|---------------|
| 准确性     | 高精度翻译    |
| 格式保持   | 完整保留原格式|

> 注意:翻译时会严格保持原文的语义和风格。

`代码示例`: console.log("Hello World");

### 结构化工具
`StructuredTool.from_function` 类方法比 `@tool` 装饰器提供了更多可配置性，且无需编写太多额外代码。

In [6]:
from langchain_core.tools import StructuredTool


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(func=multiply, coroutine=amultiply)

print(calculator.invoke({"a": 2, "b": 3}))
print(await calculator.ainvoke({"a": 2, "b": 5}))

6
10


要配置它：

In [7]:
class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    return_direct=True,
    # coroutine= ... <- you can specify an async method if desired as well
)

print(calculator.invoke({"a": 2, "b": 3}))
print(calculator.name)
print(calculator.description)
print(calculator.args)

6
Calculator
multiply numbers
{'a': {'description': 'first number', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'second number', 'title': 'B', 'type': 'integer'}}


## 从Runnable创建工具
LangChain 中的 [Runnables](/docs/concepts/runnables)（可接受字符串或 `dict` 类型输入）可通过 [as_tool](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable.as_tool) 方法转换为工具，该方法支持为参数指定名称、描述以及额外的模式信息。
示例用法：

In [8]:
from langchain_core.language_models import GenericFakeChatModel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("human", "Hello. Please respond in the style of {answer_style}.")]
)

# Placeholder LLM
llm = GenericFakeChatModel(messages=iter(["hello matey"]))

chain = prompt | llm | StrOutputParser()

as_tool = chain.as_tool(
    name="Style responder", description="Description of when to use tool."
)
as_tool.args

/var/folders/4j/2rz3865x6qg07tx43146py8h0000gn/T/ipykernel_95770/2548361071.py:14: LangChainBetaWarning: This API is in beta and may change in the future.
  as_tool = chain.as_tool(


{'answer_style': {'title': 'Answer Style', 'type': 'string'}}

请参阅[本指南](/docs/how_to/convert_runnable_to_tool)了解更多详情。

## 子类化 BaseTool
您可以通过从 `BaseTool` 子类化来定义自定义工具。这种方式能提供对工具定义的最大控制权，但需要编写更多代码。

In [ ]:
from typing import Optional

from langchain_core.callbacks import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain_core.tools import BaseTool
from langchain_core.tools.base import ArgsSchema
from pydantic import BaseModel, Field


class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


# Note: It's important that every field has type hints. BaseTool is a
# Pydantic class and not having type hints can lead to unexpected behavior.
class CustomCalculatorTool(BaseTool):
    name: str = "Calculator"
    description: str = "useful for when you need to answer questions about math"
    args_schema: Optional[ArgsSchema] = CalculatorInput
    return_direct: bool = True

    def _run(
        self, a: int, b: int, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return a * b

    async def _arun(
        self,
        a: int,
        b: int,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        # If the calculation is cheap, you can just delegate to the sync implementation
        # as shown below.
        # If the sync calculation is expensive, you should delete the entire _arun method.
        # LangChain will automatically provide a better implementation that will
        # kick off the task in a thread to make sure it doesn't block other async code.
        return self._run(a, b, run_manager=run_manager.get_sync())

In [10]:
multiply = CustomCalculatorTool()
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

print(multiply.invoke({"a": 2, "b": 3}))
print(await multiply.ainvoke({"a": 2, "b": 3}))

Calculator
useful for when you need to answer questions about math
{'a': {'description': 'first number', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'second number', 'title': 'B', 'type': 'integer'}}
True
6
6


## 如何创建异步工具
LangChain 工具实现了 [可运行接口 🏃](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html)。
所有可运行对象都公开了 `invoke` 和 `ainvoke` 方法（以及其他方法如 `batch`、`abatch`、`astream` 等）。
因此，即使你只提供了工具的`sync`同步实现，仍然可以通过`ainvoke`接口来使用它，但以下是一些需要了解的重要事项：
* LangChain 默认提供了一个异步实现，该实现假设函数计算成本较高，因此会将执行委托给另一个线程。* 如果你在异步代码库中工作，应该创建异步工具而非同步工具，以避免因该线程而产生轻微的性能开销。* 若需同时支持同步和异步实现，请使用 `StructuredTool.from_function` 方法或继承 `BaseTool` 基类。* 如果同时实现同步和异步代码，且同步代码运行速度快，则覆盖LangChain默认的异步实现，直接调用同步代码即可。* 你**不能**也**不应该**在异步工具中使用同步的 `invoke` 方法。

In [11]:
from langchain_core.tools import StructuredTool


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(func=multiply)

print(calculator.invoke({"a": 2, "b": 3}))
print(
    await calculator.ainvoke({"a": 2, "b": 5})
)  # Uses default LangChain async implementation incurs small overhead

6
10


In [12]:
from langchain_core.tools import StructuredTool


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(func=multiply, coroutine=amultiply)

print(calculator.invoke({"a": 2, "b": 3}))
print(
    await calculator.ainvoke({"a": 2, "b": 5})
)  # Uses use provided amultiply without additional overhead

6
10


在仅提供异步定义时，您不应该也无法使用 `.invoke` 方法。

In [13]:
@tool
async def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


try:
    multiply.invoke({"a": 2, "b": 3})
except NotImplementedError:
    print("Raised not implemented error. You should not be doing this.")

Raised not implemented error. You should not be doing this.


## 处理工具错误
如果你正在使用带有代理的工具，很可能需要一个错误处理策略，这样代理就能从错误中恢复并继续执行。
一个简单的策略是在工具内部抛出 `ToolException`，并使用 `handle_tool_error` 指定错误处理程序。
当指定了错误处理程序时，异常将被捕获，并由该处理程序决定从工具中返回何种输出。
你可以将 `handle_tool_error` 设置为 `True`、一个字符串值或一个函数。如果它是一个函数，则该函数应接受一个 `ToolException` 作为参数并返回一个值。
请注意，仅抛出 `ToolException` 是无效的。您需要先设置工具的 `handle_tool_error` 属性，因为其默认值为 `False`。

In [14]:
from langchain_core.tools import ToolException


def get_weather(city: str) -> int:
    """Get weather for the given city."""
    raise ToolException(f"Error: There is no city by the name of {city}.")

以下是一个默认 `handle_tool_error=True` 行为的示例。

In [15]:
get_weather_tool = StructuredTool.from_function(
    func=get_weather,
    handle_tool_error=True,
)

get_weather_tool.invoke({"city": "foobar"})

'Error: There is no city by the name of foobar.'

我们可以将 `handle_tool_error` 设置为一个始终会被返回的字符串。

In [16]:
get_weather_tool = StructuredTool.from_function(
    func=get_weather,
    handle_tool_error="There is no such city, but it's probably above 0K there!",
)

get_weather_tool.invoke({"city": "foobar"})

"There is no such city, but it's probably above 0K there!"

使用函数处理错误：

In [17]:
def _handle_error(error: ToolException) -> str:
    return f"The following errors occurred during tool execution: `{error.args[0]}`"


get_weather_tool = StructuredTool.from_function(
    func=get_weather,
    handle_tool_error=_handle_error,
)

get_weather_tool.invoke({"city": "foobar"})

'The following errors occurred during tool execution: `Error: There is no city by the name of foobar.`'

## 返回工具执行的产物
有时，工具执行会产生一些产物，我们希望让链或代理中的下游组件能够访问这些产物，但又不希望将其暴露给模型本身。例如，如果某个工具返回了像文档这样的自定义对象，我们可能希望向模型传递关于此输出的某种视图或元数据，而不是将原始输出直接传递给模型。与此同时，我们可能还需要在其他地方（如下游工具中）能够访问完整的输出内容。
工具（Tool）和[工具消息（ToolMessage）](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.tool.ToolMessage.html)接口能够区分工具输出中哪些部分是为模型准备的（即ToolMessage.content），哪些部分是供模型外部使用的（ToolMessage.artifact）。
:::info 需要 ``langchain-core >= 0.2.19``
此功能已在 `langchain-core == 0.2.19` 版本中添加。请确保您的软件包已更新至最新版本。
:::
如果我们希望工具能够区分消息内容和其他附属产物，在定义工具时需要指定`response_format="content_and_artifact"`，并确保返回一个由（内容, 附属产物）组成的元组：

In [18]:
import random
from typing import List, Tuple

from langchain_core.tools import tool


@tool(response_format="content_and_artifact")
def generate_random_ints(min: int, max: int, size: int) -> Tuple[str, List[int]]:
    """Generate size random ints in the range [min, max]."""
    array = [random.randint(min, max) for _ in range(size)]
    content = f"Successfully generated array of {size} random ints in [{min}, {max}]."
    return content, array

如果我们直接使用工具参数调用我们的工具，我们将仅获取输出内容部分：

In [19]:
generate_random_ints.invoke({"min": 0, "max": 9, "size": 10})

'Successfully generated array of 10 random ints in [0, 9].'

如果我们使用工具调用（如由工具调用模型生成的那些）来调用我们的工具，我们将得到一个工具消息，其中包含工具生成的内容和产物：

In [20]:
generate_random_ints.invoke(
    {
        "name": "generate_random_ints",
        "args": {"min": 0, "max": 9, "size": 10},
        "id": "123",  # required
        "type": "tool_call",  # required
    }
)

ToolMessage(content='Successfully generated array of 10 random ints in [0, 9].', name='generate_random_ints', tool_call_id='123', artifact=[4, 8, 2, 4, 1, 0, 9, 5, 8, 1])

在子类化 BaseTool 时，我们同样可以这样做：

In [21]:
from langchain_core.tools import BaseTool


class GenerateRandomFloats(BaseTool):
    name: str = "generate_random_floats"
    description: str = "Generate size random floats in the range [min, max]."
    response_format: str = "content_and_artifact"

    ndigits: int = 2

    def _run(self, min: float, max: float, size: int) -> Tuple[str, List[float]]:
        range_ = max - min
        array = [
            round(min + (range_ * random.random()), ndigits=self.ndigits)
            for _ in range(size)
        ]
        content = f"Generated {size} floats in [{min}, {max}], rounded to {self.ndigits} decimals."
        return content, array

    # Optionally define an equivalent async method

    # async def _arun(self, min: float, max: float, size: int) -> Tuple[str, List[float]]:
    #     ...

In [22]:
rand_gen = GenerateRandomFloats(ndigits=4)

rand_gen.invoke(
    {
        "name": "generate_random_floats",
        "args": {"min": 0.1, "max": 3.3333, "size": 3},
        "id": "123",
        "type": "tool_call",
    }
)

ToolMessage(content='Generated 3 floats in [0.1, 3.3333], rounded to 4 decimals.', name='generate_random_floats', tool_call_id='123', artifact=[1.5566, 0.5134, 2.7914])